# Generating Audio with LSTM 

This notebook will let you download a song from Youtube and model it with an **LSTM**

This models the way each spectral frame follows another and can be used to generate new raw audio

## Install prerequisites and get code

In [34]:
!pip install -U numpy==1.19.5

^^Restart runtime if required!^^

In [26]:
%tensorflow_version 1.x
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # less warnings ...
import tensorflow as tf

In [27]:
!git clone https://github.com/ual-cci/music_gen_interaction_RTML.git

fatal: destination path 'music_gen_interaction_RTML' already exists and is not an empty directory.


In [28]:
# python libraries
!pip install Pillow opencv-python PyWavelets tqdm slugify
!pip install -U Flask
!pip install lws==1.2.6
!pip install tflearn
!pip install librosa==0.7.2
!pip install numba==0.48
!pip install mock

In [29]:
%cd /content/music_gen_interaction_RTML

/content/music_gen_interaction_RTML


## Download a sample audio:

Note: replace the url with whatever music video you want - or upload a file directly ... You can use the ffmpeg to convert it to wav later.

In [30]:
# get a youtube downloader
!sudo curl -L https://www.youtube.com/watch?v=1mTsu8HvC1g /usr/local/bin/youtube-dl
!sudo chmod a+rx /usr/local/bin/youtube-dl

<!DOCTYPE html><html style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" lang="zh-Hant-TW" system-icons typography typography-spacing><head><meta http-equiv="X-UA-Compatible" content="IE=edge"/><script nonce="VAfobRlSmIvpyo4ajH7Qsg">var ytcfg={d:function(){return window.yt&&yt.config_||ytcfg.data_||(ytcfg.data_={})},get:function(k,o){return k in ytcfg.d()?ytcfg.d()[k]:o},set:function(){var a=arguments;if(a.length>1)ytcfg.d()[a[0]]=a[1];else for(var k in a[0])ytcfg.d()[k]=a[0][k]}};
window.ytcfg.set('EMERGENCY_BASE_URL', '\/error_204?t\x3djserror\x26level\x3dERROR\x26client.name\x3d1\x26client.version\x3d2.20220308.01.00');</script><script nonce="VAfobRlSmIvpyo4ajH7Qsg">(function(){window.yterr=window.yterr||true;window.unhandledErrorMessages={};window.unhandledErrorCount=0;
window.onerror=function(msg,url,line,columnNumber,error){var err;if(error)err=error;else{err=new Error;err.stack="";err.message=msg;err.fileName=url;err.lineNumber=line;if(!isNaN(columnNumber))err["colum

# Set up training

In [31]:
%cd /content/music_gen_interaction_RTML/

from unittest.mock import Mock, MagicMock
args = MagicMock(name='method')
sample_rate = 44100

# This will set the same setting for the training and the running of the model:
args.lstm_layers = 3
args.lstm_units = 128
args.sample_rate = sample_rate
args.sequence_length = 40
args.async_loading = True
args.amount_epochs = -1

/content/music_gen_interaction_RTML


# Pick a new song from Youtube 

## Insert your own url below and train 



In [8]:
youtube_url = "https://www.youtube.com/watch?v=1mTsu8HvC1g"

!youtube-dl -ci -f "/content/Zulu Tribal Dance in South Africa-HxhhF_nHxIs.m4a" $youtube_url -o '/content/Zulu Tribal Dance in South Africa-HxhhF_nHxIs.m4a'
!ffmpeg -i '/content/Zulu Tribal Dance in South Africa-HxhhF_nHxIs.m4a' -ac 2 -f wav full.wav

/usr/local/bin/youtube-dl: line 1: syntax error near unexpected token `<'
/usr/local/bin/youtube-dl: line 1: `<!DOCTYPE html><html style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" lang="zh-Hant-TW" system-icons typography typography-spacing><head><meta http-equiv="X-UA-Compatible" content="IE=edge"/><script nonce="FBSOR9zwF2yEfJI1DxoOdA">var ytcfg={d:function(){return window.yt&&yt.config_||ytcfg.data_||(ytcfg.data_={})},get:function(k,o){return k in ytcfg.d()?ytcfg.d()[k]:o},set:function(){var a=arguments;if(a.length>1)ytcfg.d()[a[0]]=a[1];else for(var k in a[0])ytcfg.d()[k]=a[0][k]}};'
/bin/bash: -o: command not found
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enab

In [9]:
# Audio file preparation - this cuts a 1 minute sample from the audio file:
!ffmpeg -y -ss 60 -i full.wav -t 60 -c copy sample.wav

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

### You will probably need to train for 300 epochs

In [11]:
number_of_epochs = 300 # will take cca 4min
number_of_epochs = 400 # will take cca 8min

song_name = "sample.wav" # < we will train the model on this wav file
model_name = "my_trained_model" # < and then save the model under this name (check the .tfl files in the folder afterwards)

# Train Model

In [12]:
# ----[keep the same bellow]-------------------------------------------------------------------

# takes time!
!python training_handler.py -target_file $song_name -model_name $model_name -amount_epochs $number_of_epochs -batch_size 512 \
                            -lstm_layers $args.lstm_layers  -lstm_units $args.lstm_units -sample_rate $args.sample_rate -sequence_length $args.sequence_length
from IPython.display import clear_output 
clear_output()

import glob
import numpy as np
i = glob.glob("*.tfl.png")
from IPython.display import Audio, Image
display(Image(i[0]))

Streaming output truncated to the last 5000 lines.
Training Step: 2016  | total loss: 3.38396 | time: 0.788s
| Adam | epoch: 184 | loss: 3.38396 - acc: 0.2361 -- iter: 1536/5208
Training Step: 2017  | total loss: 3.35440 | time: 1.092s
| Adam | epoch: 184 | loss: 3.35440 - acc: 0.2341 -- iter: 2048/5208
Training Step: 2018  | total loss: 3.31512 | time: 1.398s
| Adam | epoch: 184 | loss: 3.31512 - acc: 0.2331 -- iter: 2560/5208
Training Step: 2019  | total loss: 3.36048 | time: 1.686s
| Adam | epoch: 184 | loss: 3.36048 - acc: 0.2325 -- iter: 3072/5208
Training Step: 2020  | total loss: 3.40414 | time: 1.974s
| Adam | epoch: 184 | loss: 3.40414 - acc: 0.2329 -- iter: 3584/5208
Training Step: 2021  | total loss: 3.39238 | time: 2.266s
| Adam | epoch: 184 | loss: 3.39238 - acc: 0.2352 -- iter: 4096/5208
Training Step: 2022  | total loss: 3.37674 | time: 2.547s
| Adam | epoch: 184 | loss: 3.37674 - acc: 0.2382 -- iter: 4608/5208
Training Step: 2023  | total loss: 3.42116 | time: 2.830s
| 

# Generate

As we mentioned in the lecture, we start from a place in the original audio track, then use the model to keep predicting new audio frames. 

Unfortunately, sometimes it can get stuck in a loop, or just stop generating interesting things so we can give it a kick by jumping to a new point in the song. 

Once we've switched, we keep using the model to generate new audio frames!

Below, you can specify a sequence of lengths and changes to the generation process to create a new audio output. 

The `sequence` array stores these points. Each item in is an array that stores the **start position** and **segment length**. The default has three, but you can add in as many as you want and/or change the existing values. 

```
sequence = [
  [start1, length1],
  [start2, length2],
  [start3, length3],
  etc....
]
```


The default code:

* Starts generating 10% through the song and generates 200 frames 

* Moves to 60% through the song and generates 300 frames

* Moves to 90% through the song and generates 150 frames

**Try with your own!**

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# Here you can change the paths to other trained models / other song files
load_generative_seeds_from = song_name
load_model_from = model_name+".tfl"

In [22]:
from server_handler import ServerHandler
import settings

my_settings = settings.Settings(args)
print("Important Settings: settings.lstm_layers=", my_settings.lstm_layers, ", settings.lstm_units=", my_settings.lstm_units,
              ", settings.sample_rate=", my_settings.sample_rate)

generation_handler = ServerHandler(my_settings, manual_loading = True)
generation_handler.manual_init_song_model(load_generative_seeds_from, load_model_from)

# slightly experimental interpolation through the latents while generating ...

generation_handler.change_impulse(0.2) # set to 20% sharp

sequence = [
  #Starts generating 10% through the song and generates 200 frames 
  [0.1, 200],
  #Moves to 60% through the song and generates 300 frames
  [0.6, 300],
  #Moves to 90% through the song and generates 150 frames
  [0.9, 150],          
]

output_audio = []

for i in sequence:
  position_in_the_song = i[0]
  requested_length = i[1]
  generation_handler.change_impulse_smoothly_start(position_in_the_song) # allow interpolation
  audio, predict, reconstruct = generation_handler.generate_audio_sample(requested_length, interactive_i=position_in_the_song)
  output_audio.append(audio)

clear_output()

# Play Audio

If you want to download the audio, you can use the Colab File Explorer on the left <----. 

Find the file `generated_output_exp_concat.wav` and select **download**

In [23]:
import librosa
#\dc"Kca'lms'M
import numpy as np

In [24]:
from IPython.display import Audio, Image
output_audio = np.concatenate(output_audio)

out_name = 'generated_output_exp_concat.wav'
librosa.output.write_wav(out_name, output_audio, sr=sample_rate)
Audio(out_name)

# Clean up if you are going to train a new model from a different youtube video

In [25]:
# fast cleanup
!mkdir unused
!mv *.wav unused/
!mv *.m4a unused/

mkdir: cannot create directory ‘unused’: File exists
mv: cannot stat '*.m4a': No such file or directory
